In [ ]:
%run ../functions.ipynb

## 拉對比

In [ ]:
def swift_algo(gray, Lambda=3, Gamma=2):
    ## Step 1: Image normalization
    norm = gray/255
    
    ## Step 2: Comupte u series from normalized image
    u = np.log(1+norm)
    u_avg = np.mean(u)
    u_std = np.std(u)
    
    ## Step 3: Compute ...
    v = np.power(u, Lambda)/Lambda
    f = np.power( np.exp( (np.tan(u)-u_std) / (np.exp(u)-v)), Gamma)
    
    ## Step 4: Redefine distrubution
    t = (f-np.min(f)) / (np.max(f)-np.min(f))
    final = (t*255).astype('uint8')
    
    return final

## 影像處理

In [ ]:
def image_processing(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5,5), 0)
    nlm  = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
    _, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    edge = cv2.Canny(otsu, 0, 20)
    
    return blur, nlm, otsu, edge

def get_edge_points(edge_img):
    edge_points = [[j,i] for i, r in enumerate(edge_img) for j, c in enumerate(r) if c == 255]

    return edge_points

## 顯示結果

In [ ]:
def show_resoult(predictions):
    mean = np.mean(predictions, axis=0)
    var  = np.var(predictions,  axis=0)
    x_max, y_max = get_max_length_axil(predictions)
    print("-----------------------------------------------------")
    print("Statistics: ")
    print("Mean Centroid: ({:.5f}, {:.5f})".format(mean[0], mean[1]))
    print("Variance     : ({:.5f}, {:.5f})".format(var[0], var[1]))
    print("Max Length   : {:.5f}, {:.5f} (pixels)".format(x_max, y_max))

## 亞像素座標

In [ ]:
def sigmoid(x, a, b):
    return 1.0 / (1.0 + np.exp(-a*(x-b)))

def get_influential_neighbors(coord):
    ## 依照 sample 的點落在當下 pixel 中的哪個空間，分為 4 個case
    ##     0 . 1
    ##    .......
    ##     2 . 3
    x = round(coord[0])
    y = round(coord[1])
    a = y > coord[0] and 1 or 0
    b = x > coord[1] and 1 or 0
    space = np.dot([1,2], [a,b])
    
    switcher = {
        0: [[y,x], [y-1,x], [y-1,x-1], [y,x-1]],
        1: [[y,x], [y,x+1], [y-1,x+1], [y-1,x]],
        2: [[y,x], [y,x-1], [y+1,x-1], [y+1,x]],
        3: [[y,x], [y+1,x], [y+1,x+1], [y,x+1]],
    }
    
    neighbors_loc = np.asarray(switcher.get(space)).astype(int)
    return neighbors_loc

def get_negihbors_val(neighbors_loc, img):
    neighbors_val = [img[y,x] for y, x in neighbors_loc]
    return neighbors_val

def get_negihbors_wt(coord, neighbors_loc): 
    ## 取的每個 neighbor 與 sample 點的距離
    neighbors_dist = LA.norm(coord - neighbors_loc, axis=1)
    neighbors_dist = neighbors_dist / np.sum(neighbors_dist)

    ## 若與第一個 pixel 的距離是 0 (即在 pixel 上)則為當下的值
    ## 否則每個 neighbor 的權重為距離倒數
    neighbors_wt = neighbors_dist[0] == 0 and [1,0,0,0] or (1 - neighbors_dist)/3
    return neighbors_wt

def get_spl_val(coord, img):
    ## 取得有影響力的 4 個相鄰點及相對應的 值 和 權重
    neighbors_loc = get_influential_neighbors(coord)
    neighbors_val = get_negihbors_val(neighbors_loc, img)
    neighbors_wt  = get_negihbors_wt(coord, neighbors_loc)
    
    ## 計算 sample 的值
    value = np.dot(neighbors_val, neighbors_wt)  
    return value

def get_sbp_edges(center, edge_pts, img):

    sbp_edges = []
    for pt in edge_pts:
        ## 取得中心到邊緣的單位向量
        c2e_V = pt - center
        c2e_V = c2e_V/LA.norm(c2e_V)
        
        ## 取得 sapmple 點的'座標'及'亮度值' 
        spl_pts  = [pt + i*c2e_V for i in range(-5,6)]       # 需要再找道合適參數
        spl_vals = [get_spl_val(spl_pt, img) for spl_pt in spl_pts]

        ## 擬合 S 曲線
        x_label = np.asarray(range(0,11))
        y_vals  = (spl_vals - min(spl_vals))/ (max(spl_vals) - min(spl_vals))
        popt, pcov = curve_fit(sigmoid, x_label, y_vals)
        
        ## 取得 Suboixels' 的位置
        sbp = pt - 1.2*(5-popt[1])*c2e_V
        sbp_edges.append(sbp)
        

    return sbp_edges